In [3]:
import pandas as pd

label_column = "Survived"

X_test = pd.read_csv("./data/test.csv")

raw_train = pd.read_csv("./data/train.csv")
y_train = raw_train["Survived"]

In [4]:
raw_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
X_train = raw_train.copy()
has_cabin = (X_train["Cabin"].map(lambda cabin: 1 if str(cabin) != "nan" else 0))
X_train = X_train.drop("Cabin", axis=1)
X_train["Cabin"] = has_cabin

In [6]:
is_female = (X_train["Sex"].map(lambda sex: 1 if str(sex) != "female" else 0))
X_train = X_train.drop("Sex", axis=1)
X_train["Sex"] = has_cabin

In [7]:
columns = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Cabin", "Embarked"]
X_train = X_train[columns]

In [8]:
X_train.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,22.0,1,0,7.2500,0,S
1,1,1,1,38.0,1,0,71.2833,1,C
2,1,3,0,26.0,0,0,7.9250,0,S
3,1,1,1,35.0,1,0,53.1000,1,S
4,0,3,0,35.0,0,0,8.0500,0,S


In [9]:
corr_matrix = X_train.corr()

In [10]:
corr_matrix["Survived"].sort_values(ascending=False)

Survived    1.000000
Sex         0.316912
Cabin       0.316912
Fare        0.257307
Parch       0.081629
SibSp      -0.035322
Age        -0.077221
Pclass     -0.338481
Name: Survived, dtype: float64

In [11]:
final_columns = ["Cabin", "Fare", "Sex", "Pclass", "Age"]
X_train_final = X_train[final_columns]

In [12]:
X_train_final.head()

,Cabin,Fare,Sex,Pclass,Age
0,0,7.2500,0,3,22.0
1,1,71.2833,1,1,38.0
2,0,7.9250,0,3,26.0
3,1,53.1000,1,1,35.0
4,0,8.0500,0,3,35.0


In [13]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()

In [15]:
 num_scaled = min_max.fit_transform(X_train_final[["Fare", "Age"]])

In [17]:
num_scaled

array([[0.01415106, 0.27117366],
       [0.13913574, 0.4722292 ],
       [0.01546857, 0.32143755],
       ...,
       [0.04577135,        nan],
       [0.0585561 , 0.32143755],
       [0.01512699, 0.39683338]])

In [ ]:
from sklearn.compose import ColumnTransformer
col_trans = ColumnTransformer([
    ("num", min_max,["Fare", "Age"])
])